**Lecture link:** https://www.scaler.com/meetings/i/dsml-advanced-gbdt-and-xgboost/archive

## Content

1. Recap (10:30 - 19:30)
2. Questions(19:30- 46:20)
3. GBDT (55:30 - 1:48:35)
4. Xgboost (1:48:35 - 2:16:00)
5. LightBGM (2:16:00 - 2:26:15)
6. Adaboost (2:26:15 - 2:37:00)

## Recap

**What is pseudo residual?**

we have studied this in previous lecture let's quickly recap that.

* Let us assume we have $n$ number of data points
and trained $m-1$ classifiers then the pseudo residual of $m^{th}$ classifier is given by differentiation of loss function with respect to final model of $(m-1)^{th} $  classifier
 * $res_{i,m} = [\frac{d(L(y_i,F(x_i))}{d(F(x_i))}]_{(F_{m-1}(x_i))}$



<img src='https://drive.google.com/uc?id=1N89WqSgoLwaTw4TOXEf1ski0jqm2XNwO' >

Now, let's assume our loss is squared loss
* Therefore our loss becomes
 * $L(y_i,F(x_i)) = (y_i - F(x_i))^2$
* now it's derivative is
 * $\frac{d(L)}{d(F_m(x_i))} = - 2 (y_i - F_m(x_i)) $

**How do we compute the pseudo residual?**

By comparing the above two equations we can say that
* $res_{i,m} = 2 (y_i - F_m(x_i))_{F_{m-1}x_i} $




<img src='https://drive.google.com/uc?id=1LT91ZmqU4pysZf337xqdwnsLJBYB36hX' >

**What do we already have?**
* We already have our data $(x_i,y_i)_{i=1}^n$
* We also have the function $F_{m-1}(x_i)$

So now all we have to do is to pass the $x_i$ in the function and use the values of $y_i$ to find the residual  $res_{i,m}$

Now we train our $m^{th}$ model on $(x_i, res_{i,m})$
where $res_{i,m}$ is the residual of $(m-1)^{th}$ model and it is used to train the $m^{th}$ model

* Here the key idea is that pseudo residul is equal to the negative of the differentiation of the loss function with respect to final output of the previous model
* So, for any loss function given which is differentiable, we can easily compute our residual




<img src='https://drive.google.com/uc?id=1JuRzFFhW3v7tafBl9pbLGsmAsKGcrhKk' >



## Questions

**When are $α$ calculated?**

* Firstly, let's use $γ$ instead of $α$ from now on
* So our $F_m(x_i) = γ_0 h_0(x) + γ_1 h_1 (x) +......+ γ_m h_m(x)$
* In the $m^{th}$ iteration when we are training the $h_m(x)$ we also compute the $γ$ value by using a simple optimization.

**What happens to the residuals in the test time?**

We use the puedo residuals only to train the base learners $h(x_i)$ whixh are shallow trees in GBDT to get the $y$
* So in run time when a query point $x_q$ is given we just plug in the value of $x$ in eaxh our base learners and get the output.
 * $∑_{i=0}^M r_i h_i(x_q) = ŷ$


<img src='https://drive.google.com/uc?id=1PaBn5nxOlKr859RdZPoVtiSsKvZo1UBc' >


**What is the benifit of training our base learners on the errors of the previous models?**

Lets us assume $x_i$ is a 4 dimensional vector $x$ = [0,1,2,0] and $y_i$ be 26.2 and all $γ=1$

* after training ,our first or mean model let the value be 12,i.e the model $h_0(x_i)$
* we know $F_m(x_1) = γ_0 h_0(x) + γ_1 h_1 (x) +......+ γ_m h_m(x)$, so the sum of all other models except the mean model should be 14.2.
* So, here after every iteration as we are tarining on the residuals of the previous model,  the next models train on lesser and lesser values


<img src='https://drive.google.com/uc?id=1s1FLz84eW4L97hFUka3IOAC0q3Jxe0bE' >




**What if residual becomes zero?**

Here **$m$ is our hyper-parameter**, so, we have to control $m$ and stop the model learning after the residual is minimum, or the model will **overfit**.

**How does this work for classification?**

Let us assume we have the data ($x_i,y_i$),but our values of $y_i$ be 0 or 1.
* Now, we have the predict the probability of $y_i$ being 0 or 1 i.e $p_i$
* for model $F_m(x_i)$ we want to predict $p_i$
* The loss-function for the classification is log-loss
 * We can prove that $\frac{d(L(y_i,F(x_i))}{d(F(x_i))} = (p_i - y_i)$

* For model where $x_i$ = [0,1,3,1] and $y_i=1$
  * let $h_0(x_1)$ be 0.4
  * The sum of other models should be 0.6 ad so on it continues just like we saw in regression.

When the $y_i$=0
* The models after mean model tries to predict negative values so that final output becomes 0.




<img src='https://drive.google.com/uc?id=1I2c-bxsgVreoPxR04gj46s_G0hcLUODj' >


<img src='https://drive.google.com/uc?id=1u-Oo_5Ofzn7hrmRCBScUddd-_EwXYWGg' >


<img src='https://drive.google.com/uc?id=1LSTGiUi69qIY3_XjEe99mJJdOJcIU5_d' >


<img src='https://drive.google.com/uc?id=1pjZuf3zDLeY60NHPHOrb-IKYddoNRGSv' >



## GBDT

### GBDT Algorithm

* Here we have 3 inputs
  * The data set, $(x_i,y_i)_{i=1}^n$
  * A differentiable loss function $L(y_i, F(x))$
  * The number of iterations $M$

Let's see the steps in the algorithm


1. Initialize the model $F_0(x) = arg\ min_γ ∑_{i=1}^n L(y_i,γ)$
  * That is initialising the model with $γ$ which minimises the sum of loss function across all  values
     * This will be mean for sqaured loss
     * and $p(y=1)$ across the dataset for log-loss

 * Let's consider for sqaured loss
    * $F_0(x)= arg\ min_γ ∑_{i=1}^n (y_i-γ)^2$
    * Here the only variable is $γ$, so minimize that using Gradient Descent
    * Therefore, differentiation of the loss function with $γ$ gives $\frac{d g(γ)}{dγ} = ∑_{i=1}^n -2 (y_i-γ)$ which should we equated to 0, to get minimum
       *  $∑_{i=1}^n y_i = n γ$
       * $ \frac{1}{n}∑_{i=1}^n y_i= γ$



2. Now, we have to iterate $M$ times and build $M$ base learners beyond 1st base learner
* So, the very first step is to calculate psuedo residuals
  * $res_{i,m} = [\frac{d(L(y_i,F(x_i))}{d(F(x_i))}]_{(F_{m-1}(x_i))}$ for all $i$ 1 to n

* Now, train a base learner $h_m(x)$ on $(x_i, r_{i,m})_{i=1}^n$

* Now compute $γ_m$ by using the one-dimensional optimization problem, which uses the loss function to find the γ, so that loss function is minimum.
 * $γ_m= arg\ min_γ ∑_{i=1}^n L(y_i , F_{m-1}(x_i)) + γh_m(x_i)$
* Now update the model as
 * $F_m(x) = F_{m-1}(x) + γ_mh_m(x)$






3. The final output is the model $F_M(x).$



<img src='https://drive.google.com/uc?id=1jlwNnKXWO2pCem4M4XWdYdKpTvqGIDKD' >

<img src='https://drive.google.com/uc?id=12e63puZrN4pke50UEqaLXL6yQx1TRokM' >




<img src='https://drive.google.com/uc?id=1aSNgvc4Zmvwsr1rADhNf8zM15dfPR_d-' >


### Overfitting-Underfitting Trade-off

We know that Regularization tries to avoid overfitting of the model.

**How can we Regularize the GBDT model?**

**What are the hyper-parameters we can use?**
 * $M$ - The number of base learners
  * As $M$ increases, the model will overfit

* The depth of base learners
 * As depth increases,model overfit

The another method of regularization is
 * Shrinkage/ Regularizer/ Learning-rate



<img src='https://drive.google.com/uc?id=1OehWNYBxUvBszDv8-VGFKaS4aefZbVZH' >

* In shrinkage all we do is to add a regularization term $𝜈$
in the model of $F_m(x)$
 * $F_m(x) = F_{m-1}(x) +𝜈\ γ_mh_m(x)$
 * This 𝜈 always lies between 0 and 1, this is a way to scale γ, which is a hyper-parameter

**What happens as 𝜈 increases?**

* Let's see what happens when **$𝜈=0$**, we will have only $h_0(x)
$ which is a very underfit model

but **as 𝜈 increases, chances of overfitting also increases**

* We have **same 𝜈 for all models in GBDT**

<img src='https://drive.google.com/uc?id=1CSNOZVJYp3NyRSjMTxeMj06Quve82fD9' >



<img src='https://drive.google.com/uc?id=1zTQ_3nJ8nWZgUzCVTjJuAUoY_uQ-qY_N' >



**What should be done to $M$ if 𝜈 is decreased to keep the performance same?**

* Number of **base learners should be increased**, because as we are decresing 𝜈 we are restricting base learners contribution due to which the error increases


We use 𝜈 even when the $γ$ is there to avoid overfit, as there are high chances of overfitting

**Can we add a regulariser to γ?**

* Yes we can add regularisation to vector $γ$ whixh is
$γ$=[$γ_1,γ_2,γ_3....γ_M$]
* We can even add other terms controlling the number of leaf nodes which also becomes hyper parameter







<img src='https://drive.google.com/uc?id=1EtZi-SVT7_ARPEuI2cxJ1QNOAQl0GVYJ' >

### Stochastic Gradient Boosting

* The stochhastic gradient boosting follow the same framework as GBDT, but it also does row sampling and column sampling
* We do this sampling because it's **randomisation as regularisation**
* The libraries like Xgboost uses this.




<img src='https://drive.google.com/uc?id=1b0t8tmqt6ksDPEK2Uqe-Q01IXUFPmSfC' >

### Train time

* We saw that Random forest is trivially parallelizable.
 because we could train them on different cores or system

* But GBDT is the sequential model, Hence it is slow to train

**How can we speed up GBDT?**

* Row sampling and column sampling can speed up the model a little
* While building each base leraners we can apply **parallelisation**, i.e one sub-tree can be built on a core and the other sub-tree can be built on another core
* Even for bulding a single tree, **we have to calculate entropy of many features, this can be parallelised**





<img src='https://drive.google.com/uc?id=12V2N2vzcO9mBSj40dWX_xqv_5QezuJYK' >

## Xgboost

* Xgboost have a lot of code and algorithms optimized
 * Example : if you have numerical feature $f_i$, instead of tryinig all the values for thresholding, it build a histogram of data and use simple rules like quartiles and percentiles to make thresholding.
* It also does multi core optimization.

### Xgboost library terms

* **Eta** or the learning rate is the shrinking/regularization term which we studier earlier
* **min_split_loss** this gives the minimum Information Gain which you want for further split.

**What happens if min_split_loss increases?**
* If the min_split_value of the model is indreased, the splitting stops if the min_split_loss is not met. Due to this the depth of the decreases resulting in shallow tress, which results in the underfitting of the model.






<img src='https://drive.google.com/uc?id=1tvyj1KYedvVhaeIJIAh_ZillGmf5p5rE' >

* **max_depth**, this parameter is used to set the depth of the base learners  




<img src='https://drive.google.com/uc?id=1k_JD_dg0las0-N03UWQ_jSzkIP1M-b9W' >

* **min_child_weight** you can increase the weight of the child due to which the splitting stops if the erquired threshold is not met

* **subsample**. it's nothing but the row sampling
 * There are many ways of sampling, same with column sampling

* **Lambda**, it is the used apply the L2 regularization on the weights, which is $γ$ in our case



<img src='https://drive.google.com/uc?id=1Vvz_rxFrx0EAgwClXn8OFwkhZ_5eD9C0' >

* **Tree_methods**, there are many ways to build a tree, we can specify the method basing on the conditions




<img src='https://drive.google.com/uc?id=19vAWSdAFmHYcIh5pWgxPLneqyD8Eze1d' >

* **n_estimator** in scikit learn API, which sets the number of base learners in the model

* These are some of the most used parameters



<img src='https://drive.google.com/uc?id=1n0JuWHd8NerqKnYnWFQqIwKeNuJG7PUr' >

**how to implement random forest using Xgboost?**

There few terms which are to be learnt like
 * **booster** which is to be set to gbtree as we are using a tree based model
 * **subsample**, this is used for row sampling and also a little column sampling
 * **num_parallel_tree** gives the number of parallel trees can be built, like while building a model $h(x)$, multiple ways of the same model can be built in seperate cores.
  * if u set these value to 100,  100 trees are parallely built in random forest , and should also set **num_boostinig_roounds=1**,
  so that only one base learner is built in 100 ways **



<img src='https://drive.google.com/uc?id=1FS9Y5K8mhDawpJUMSaLV5KEbD0muHpPW' >



<img src='https://drive.google.com/uc?id=10mps_xf9i7cpGiRM-CjK5qhso57O2a7u' >

## LightGBM

It was built at microsoft, primarily for a faster GBDT

It is typically faster than Xgboost because of the more code optimization
  * One such strategy is **GOSS - Gradient based one side sampling**
    * When we are building the $m^{th}$ model the points we have is $ x_i,res_{i,m} $, so here instead of considering all points we drop the points in which the $res_{i,m}$ is small along with a  sampling model where probability of getting large residual value is higher
    * Here the key is to reduce the number of data points due to which the model becomes faster
  



 <img src='https://drive.google.com/uc?id=1NGKKYLSyz8MzFKQtzxnh9PTUbGQoM9JP' >

* The other strategy is **Exclusive Feature Bundliing**
   * Let us assume we have a categorical feature with 3 categories and if we do **one hot encoding**, for each row only one of them will always be set.
   * so, what exclusive feature bundling does is it looks at all the dimensions and look at feature pairs if they are exclusive and group these features and then do target encoding
   *  So, here the key objective of EFB is to reduce the number of features

 <img src='https://drive.google.com/uc?id=1G2Ilv5_nhQ0rrX0KUp60v9UoqlDApWl4' >


## AdaBoost

AdaBost means Adaptive Boosting
* We prefer GBDT over AdaBoost beacuse of the flexibility of choosing loss-function

**What does AdaBoost do?**

Let us assume a toy dataset with negatives and positives
* At first we have a weak classifier which is a horizontal or a vertical plane




<img src='https://drive.google.com/uc?id=1Og9gvnSUpx3uqaS12NUybZatrwLWhIqZ' >


* $ϵ$ is the error of the model

* $α$ is how good our model is, whixh is considered as the weight of the model


<img src='https://drive.google.com/uc?id=1e17PWvIvvhH0NeL_hAB45iFI8pBZAnPt' >



* After building the first model, we give more weightage to the erronous data while building the second model
* Now as more weightage is given to the erronous data now the second model will be based on the erronous data


<img src='https://drive.google.com/uc?id=1EauFehFWBMZB7scQaBKgv9bcTYwv9N_f' >



* Again after the second model there will be few erronous points in the classification, so these points will be given more weightage while building the next model
* so, here what we are doing is giving weights to points proportional to the errors they are making
* So, to get final model, we multiply each model with a weight and combine them

<img src='https://drive.google.com/uc?id=1PX8ikROK910A_ms432Wfm6CB0vt7yg9H' >


